# Identifying job sets that have not progressed and cancel them
---

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pandas as pd

from methods import (
    get_df_jobs,
    get_df_jobs_data,
    get_df_jobs_anal,
    )

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/dft_workflow/job_analysis/id_resource_waste


# Read Data

In [2]:
df_jobs = get_df_jobs()

df_jobs_data = get_df_jobs_data()

df_jobs_anal = get_df_jobs_anal()

In [3]:
# Removing systems that were marked to be ignored
from methods import get_systems_to_stop_run_indices

indices_to_stop_running = get_systems_to_stop_run_indices(df_jobs_anal=df_jobs_anal)

# df_jobs_anal = df_jobs_anal.drop(index=indices_to_stop_running)
# df_resubmit = df_jobs_anal

# Main Loop

In [4]:
from misc_modules.pandas_methods import drop_columns

# print(list(df_jobs_anal.columns))

# ['job_id_max', 'timed_out', 'completed', 'brmix_issue', 'job_understandable', 'decision', 'dft_params_new', 'job_completely_done']
cols_to_keep = [
    "job_id_max",
    "job_completely_done",
    ]
df_jobs_anal_i = drop_columns(
    df=df_jobs_anal,
    columns=cols_to_keep,
    keep_or_drop="keep",
    )

In [5]:
# assert False

In [6]:
index_keys = list(df_jobs_anal_i.index.names)

# #########################################################
data_dict_list = []
# #########################################################
for index_i, row_i in df_jobs_anal_i.iterrows():
    # #####################################################
    data_dict_i = dict()
    # #####################################################
    compenv_i, slab_id_i, ads_i, active_site_i, att_num_i = index_i
    # #####################################################
    job_completely_done_i = df_jobs_anal.job_completely_done
    # #####################################################


    index_dict_i = dict(zip(index_keys, index_i))

    df = df_jobs
    df = df[
        (df["compenv"] == compenv_i) &
        (df["slab_id"] == slab_id_i) &
        (df["ads"] == ads_i) &
        (df["active_site"] == active_site_i) &
        (df["att_num"] == att_num_i) &
        [True for i in range(len(df))]
        ]
    df_jobs_i = df

    num_revs_i = df_jobs_i.shape[0]

    # #####################################################
    data_dict_i.update(index_dict_i)
    data_dict_i.update(row_i.to_dict())
    # #####################################################
    data_dict_i["num_revs"] = num_revs_i
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################

# #########################################################
df = pd.DataFrame(data_dict_list)
df = df.sort_values("num_revs", ascending=False)
df = df.set_index([
    "compenv", "slab_id", "ads", "active_site", "att_num", 
    ], drop=False)
df = df.drop(labels=indices_to_stop_running)
# #########################################################

df.iloc[0:2]

compenv      slab_id ads  \
compenv slab_id     ads active_site att_num                            
slac    tefovuto_94 oh  16.0        0          slac  tefovuto_94  oh   
        fagumoha_68 oh  62.0        0          slac  fagumoha_68  oh   

                                            active_site  att_num   job_id_max  \
compenv slab_id     ads active_site att_num                                     
slac    tefovuto_94 oh  16.0        0                16        0  rilihalu_58   
        fagumoha_68 oh  62.0        0                62        0  niduvupu_51   

                                             job_completely_done  num_revs  
compenv slab_id     ads active_site att_num                                 
slac    tefovuto_94 oh  16.0        0                      False        18  
        fagumoha_68 oh  62.0        0                       True        11

In [7]:
df_i = df[df.job_completely_done == False]
df_i = df_i[df_i.num_revs > 3]

df_i

compenv      slab_id   ads  \
compenv  slab_id     ads  active_site att_num                                
slac     tefovuto_94 oh   16.0        0            slac  tefovuto_94    oh   
sherlock momaposi_60 oh   50.0        0        sherlock  momaposi_60    oh   
         kapapohe_58 o    NaN         1        sherlock  kapapohe_58     o   
         kamevuse_75 bare 49.0        2        sherlock  kamevuse_75  bare   

                                              active_site  att_num  \
compenv  slab_id     ads  active_site att_num                        
slac     tefovuto_94 oh   16.0        0                16        0   
sherlock momaposi_60 oh   50.0        0                50        0   
         kapapohe_58 o    NaN         1               NaN        1   
         kamevuse_75 bare 49.0        2                49        2   

                                                job_id_max  \
compenv  slab_id     ads  active_site att_num                
slac     tefovuto_94 oh   16.0        0        rilihalu_58   
sherlock momaposi_60 oh   50.0        0        kobemiri_55   
         kapapohe_58 o    NaN         1        tehatevi_12   
         kamevuse_75 bare 49.0        2        wumuduwi_69   

                                               job_completely_done  num_revs  
compenv  slab_id     ads  active_site att_num                                 
slac     tefovuto_94 oh   16.0        0                      False        18  
sherlock momaposi_60 oh   50.0        0                      False         5  
         kapapohe_58 o    NaN         1                      False         4  
         kamevuse_75 bare 49.0        2                      False         4

In [8]:
# df_i.iloc[0:3].index.tolist()

df_i.iloc[0:3].index.tolist()

[('slac', 'tefovuto_94', 'oh', 16.0, 0),
 ('sherlock', 'momaposi_60', 'oh', 50.0, 0),
 ('sherlock', 'kapapohe_58', 'o', 'NaN', 1)]

In [9]:
# keys = list(df_jobs_anal.index.names)
# index_dict_i = dict(zip(keys, index_i))

In [10]:
# index_dict_i

In [11]:
# data_dict_i

In [12]:
# df_jobs_anal.loc[
#     ('sherlock', 'tofiwadi_49', 'oh', 47.0, 1)
#     ]

In [13]:
# indices_to_stop_running[0:2]